# Colab BLIP Inference Server + Auto‑Register
Run BLIP on Colab, expose via Cloudflare Quick Tunnel, and automatically register with the Rust captioner.

In [ ]:
# Switch runtime to GPU: Runtime → Change runtime type → GPU
# Install dependencies
!pip -q install fastapi uvicorn transformers pillow accelerate torch cloudflared requests > /dev/null

In [ ]:
# Start the BLIP server (uses base model by default).
# To use a fine-tuned model, upload it and set MODEL_DIR accordingly.
import os, subprocess
os.environ['MODEL_DIR'] = os.getenv('MODEL_DIR', 'Salesforce/blip-image-captioning-base')
proc = subprocess.Popen(['python','-m','uvicorn','tools.blip_infer_server.server:app','--host','0.0.0.0','--port','8001'],
                        stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print('BLIP server PID', proc.pid)


In [ ]:
# Start Cloudflare quick tunnel and capture the public URL
import re, time, subprocess
t = subprocess.Popen(['cloudflared','tunnel','--no-autoupdate','--url','http://127.0.0.1:8001'],
                    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
public_url = None
for _ in range(180):
    line = t.stdout.readline().strip()
    if not line:
        time.sleep(1);
        continue
    m = re.search(r'https://[\w.-]+\.trycloudflare\.com', line)
    if m:
        public_url = m.group(0)
        break
print('PUBLIC_URL =', public_url)
print('Keep this cell running (or next cell) to keep the tunnel alive.')


In [ ]:
# Auto‑register with the captioner and refresh periodically
# Set these in the notebook before running or inline here:
# %env CAPTIONER_REGISTRY_URL=https://alttext-auto.fly.dev
# %env CAPTIONER_REGISTRY_TOKEN=YOUR_SECRET   # optional but recommended
# %env PROVIDER=colab
import os, time, requests
reg = os.getenv('CAPTIONER_REGISTRY_URL', '').rstrip('/')
tok = os.getenv('CAPTIONER_REGISTRY_TOKEN')
provider = os.getenv('PROVIDER','colab')
if not reg or not public_url:
    print('Set CAPTIONER_REGISTRY_URL and ensure tunnel started (public_url present).')
else:
    headers = {'x-registry-token': tok} if tok else {}
    def register(ttl=900):
        url = f"{reg}/v1/register"
        body = {'endpoint': public_url, 'provider': provider, 'ttl_secs': ttl}
        r = requests.post(url, json=body, headers=headers, timeout=10)
        print('register', r.status_code, public_url)
    register()
    while True:
        time.sleep(600)
        try:
            register()
        except Exception as e:
            print('register failed:', e)
